In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
# from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from typing_extensions import Optional
from data_display.input_data_plots import plot_basin_height_volume_table

from data_display.baseline_plots import *
from utility.pyomo_preprocessing import extract_optimization_results, linear_interpolation_for_bound, arange_float, linear_interpolation_using_cols 
from data_federation.input_model import SmallflexInputSchema
from utility.pyomo_preprocessing import *
from config import settings
from utility.general_function import pl_to_dict, pl_to_dict_with_tuple, build_non_existing_dirs, generate_log
from pyomo_models.input_data_preprocessing import (
    generate_baseline_index, generate_clean_timeseries, generate_water_flow_factor, generate_basin_volume_table,
    clean_hydro_power_performance_table, generate_hydro_power_state, split_timestamps_per_sim, generate_second_stage_state
)
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure

from plotly.subplots import make_subplots

import networkx as nx
from pyomo_models.baseline.baseline_input import BaseLineInput
from pyomo_models.baseline.first_stage.first_stage_pipeline import BaselineFirstStage
from pyomo_models.baseline.second_stage.second_stage_pipeline import BaselineSecondStage
COLORS = px.colors.qualitative.Plotly

log = generate_log(name="test")

In [2]:
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = "/home/ltomasini/gruobi_license/gurobi.lic"


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))

sim_setting: dict = {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True, "with_penalty": True}
baseline_input = BaseLineInput(
    input_schema_file_name=output_file_names["duckdb_input"],
    real_timestep=timedelta(hours=1),
    year=2020,
    hydro_power_mask = c("name").is_in(["Aegina hydro"]),
    volume_factor=1e-6,
)
baseline_first_stage = BaselineFirstStage(
    baseline_input, timestep=timedelta(days=1), turbine_factor=0.8)
baseline_first_stage.solve_model() 



Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 12/12 [00:01<00:00,  9.83it/s]
Solving first stage optimization problem: 100%|██████████| 1/1 [00:25<00:00, 25.46s/it]


In [4]:
sim_setting = {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True, "with_penalty": True}
baseline_second_stage = BaselineSecondStage(
    input_instance=baseline_input, 
    first_stage=baseline_first_stage, 
    timestep=timedelta(days=4), 
    log_solver_info=False,
    time_limit=20,
    **sim_setting
    )
baseline_second_stage.solve_model()





Solving second stage optimization problem:   0%|          | 0/92 [00:00<?, ?it/s]

Solving second stage optimization problem: 100%|██████████| 92/92 [01:25<00:00,  1.08it/s]


In [5]:
baseline_second_stage.simulation_results["income"].sum()

1220873.9657944546

In [6]:
baseline_second_stage.calculated_feasibility()

ColumnNotFoundError: "volume_buffer" not found

In [ ]:
power_performance_table = baseline_first_stage.power_performance_table

state_index: pl.DataFrame = pl.DataFrame()
for data in power_performance_table: 
    power_performance: pl.DataFrame = data["power_performance"]
    y_cols: list[str] = power_performance.select(cs.starts_with("alpha")).columns
    
    segments = generate_state_index_using_errors(data=power_performance, column_list=y_cols, error_percent=1.3)
    print(segments)
    if len(segments) > 10:
        log.warning(f"{len(segments)} states found in {data["H"]} hydro power plant")
    state_performance_table = filter_by_index(data=power_performance, index_list=segments)\
    .with_columns(
        c(col).abs().pipe(get_min_avg_max_diff).alias(col) for col in power_performance.columns
    ).slice(offset=0, length=len(segments)-1) 
    print(state_performance_table)
    state_index = pl.concat([
        state_index , 
        state_performance_table.with_columns(
            pl.lit(data["H"]).alias("H"),
            pl.lit(data["B"]).alias("B"))
        ], how="diagonal")
    # Add every downstream basin


[0, 10, 19, 28, 36]
shape: (4, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ height     ┆ flow_turbi ┆ flow_pumpe ┆ electrica ┆ electrica ┆ volume    ┆ alpha_tur ┆ alpha_pum │
│ ---        ┆ ned        ┆ d          ┆ l_power_t ┆ l_power_p ┆ ---       ┆ bined     ┆ ped       │
│ struct[4]  ┆ ---        ┆ ---        ┆ urbined   ┆ umped     ┆ struct[4] ┆ ---       ┆ ---       │
│            ┆ struct[4]  ┆ struct[4]  ┆ ---       ┆ ---       ┆           ┆ struct[4] ┆ struct[4] │
│            ┆            ┆            ┆ struct[4] ┆ struct[4] ┆           ┆           ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ {2350.0,23 ┆ {2.3248,2. ┆ {2.7354,2. ┆ {7.2683,7 ┆ {12.108,1 ┆ {1.39706, ┆ {3.126419 ┆ {4.426409 │
│ 55.0,2360. ┆ 3506,2.376 ┆ 6739,2.612 ┆ .4618,7.6 ┆ 1.981,11. ┆ 2.901725, ┆ ,3.173902 ┆ ,4.482,4. │
│ 0,10.0}    ┆ 4,0.0516}  ┆ 4,-0.123}  ┆ 553,0.387 ┆ 854,

In [ ]:
filter_by_index(data=power_performance, index_list=segments).with_columns(
        c(col).abs().pipe(get_min_avg_max_diff).alias(col) for col in power_performance.columns
    )

height,flow_turbined,flow_pumped,electrical_power_turbined,electrical_power_pumped,volume,alpha_turbined,alpha_pumped
struct[4],struct[4],struct[4],struct[4],struct[4],struct[4],struct[4],struct[4]
"{2350.0,2355.0,2360.0,10.0}","{2.3248,2.3506,2.3764,0.0516}","{2.7354,2.6739,2.6124,-0.123}","{7.2683,7.4618,7.6553,0.387}","{12.108,11.981,11.854,-0.254}","{1.39706,2.901725,4.40639,3.00933}","{3.126419,3.173902,3.221385,0.094966}","{4.426409,4.482,4.53759,0.111181}"
"{2360.0,2364.5,2369.0,9.0}","{2.3764,2.39918,2.42196,0.04556}","{2.6124,2.55674,2.50108,-0.11132}","{7.6553,7.82915,8.003,0.3477}","{11.854,11.7366,11.6192,-0.2348}","{4.40639,6.43919,8.47199,4.0656}","{3.221385,3.262867,3.304349,0.082963}","{4.53759,4.591632,4.645673,0.108083}"
"{2369.0,2373.5,2378.0,9.0}","{2.42196,2.44431,2.46666,0.0447}","{2.50108,2.44524,2.3894,-0.11168}","{8.003,8.17648,8.34996,0.34696}","{11.6192,11.4984,11.3776,-0.2416}","{8.47199,10.942595,13.4132,4.94121}","{3.304349,3.344738,3.385128,0.08078}","{4.645673,4.703685,4.761697,0.116024}"
"{2378.0,2382.0,2386.0,8.0}","{2.46666,2.48625,2.50584,0.03918}","{2.3894,2.33252,2.27564,-0.11376}","{8.34996,8.50395,8.65794,0.30798}","{11.3776,11.2651,11.1526,-0.225}","{13.4132,15.8282,18.2432,4.83}","{3.385128,3.420116,3.455105,0.069977}","{4.761697,4.83128,4.900863,0.139166}"
"{2386.0,null,null,null}","{2.50584,null,null,null}","{2.27564,null,null,null}","{8.65794,null,null,null}","{11.1526,null,null,null}","{18.2432,null,null,null}","{3.455105,null,null,null}","{4.900863,null,null,null}"


In [ ]:
baseline_first_stage.index["state"].unnest(["volume"])

F,min,max,S,H,B,flow,d_flow,electrical_power,d_electrical_power,S_BH,BS,HS,HSF
u32,f64,f64,u32,i32,i64,f64,f64,f64,f64,list[i64],list[i64],list[i64],list[i64]
0,14.0021,16.4183,0,0,0,-2.3768,0.023442,-11.3508,0.046602,"[0, 0, … 0]","[0, 0]","[0, 0]","[0, 0, 0]"
1,14.0021,16.4183,0,0,0,2.47158,0.00812,8.38848,0.063745,"[0, 0, … 0]","[0, 0]","[0, 0]","[0, 0, 1]"
2,14.0021,16.4183,0,0,0,0.0,0.0,0.0,0.0,"[0, 0, … 0]","[0, 0]","[0, 0]","[0, 0, 2]"
null,0.0,1.0,1,null,1,null,null,null,null,"[null, 1, … 1]","[1, 1]","[null, 1]","[null, 1, null]"
